In [1]:
!pip install spacy

    100% |████████████████████████████████| 17.3MB 2.5MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


In [1]:
!pip install spacy


In [2]:
!python -m spacy download en_core_web_lg

    100% |████████████████████████████████| 852.3MB 51.4MB/s 
  Running setup.py install for en-core-web-lg ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [5]:
!curl -L -O http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9684k  100 9684k    0     0  1589k      0  0:00:06  0:00:06 --:--:-- 2073k


In [6]:
!unzip cornell_movie_dialogs_corpus.zip

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


In [0]:
movie_lines = {}
for line in open("./cornell movie-dialogs corpus/movie_lines.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    if len(parts) == 5:
        movie_lines[parts[0]] = parts[4]
    else:
        movie_lines[parts[0]] = ""


In [0]:
import json
responses = {}
for line in open("./cornell movie-dialogs corpus/movie_conversations.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    line_ids = json.loads(parts[3].replace("'", '"'))
    for first, second in zip(line_ids[:-1], line_ids[1:]):
        responses[first] = second

In [10]:
import random
for pair in random.sample(responses.items(), 5):
    print("A:", movie_lines[pair[0]])
    print("B:", movie_lines[pair[1]])
    print()

A: ..when you think about it, Ethan, it was inevitable..no more Cold War.  No more secrets you keep from everyone but yourself, operations you answer to no one but yourself.  Then one morning you wake up and find out the President of the United States is running the country - <i>without your permission</i>.  The son-of-a- bitch!  How <i>dare</i> he?  You realize itís over, youíre an obsolete piece of hardware not worth upgrading, youíve got a lousy marriage and sixty-two grand a year.  Kittridge, weíll go after that no good son-of-a-bitch, big time!
B: We donít have to, Jim.  Heíll come after us.

A: How so?
B: By loving you, with no hope of possessing you.

A: I can't wait to see the pictures.
B: It was incredible. The light, the crowd, everything.

A: I don't care.
B: Well, I do.

A: Minute flecks were spattered across the seat and steering wheel. The instrument panel was buckled inwards, cracking the clock and the speedometer dials. The cabin was deformed, and there was dust and gla

In [11]:
import numpy as np
def sentence_mean(nlp, s):
    if s == "":
        s = " "
    doc = nlp(s, disable=['tagger', 'parser'])
    return np.mean(np.array([w.vector for w in doc]), axis=0)
sentence_mean(nlp, "This... is a test.").shape

(300,)

In [12]:
!pip install simpleneighbors

    100% |████████████████████████████████| 645kB 12.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/77/cb/7a/6f3ed44099e394e0cb0b6b41213b61fe6595b726530744f2ce
Successfully built annoy


In [14]:
from simpleneighbors import SimpleNeighbors

nns = SimpleNeighbors(300)
for i, line_id in enumerate(random.sample(list(responses.keys()), 10000)):
    # show progress
    if i % 1000 == 0: print(i, line_id, movie_lines[line_id])
    line_text = movie_lines[line_id]
    summary_vector = sentence_mean(nlp, line_text)
    if np.any(summary_vector):
        nns.add_one(line_id, summary_vector)
nns.build()

0 L644885 ...got tickets for the Knicks tonight. Go out and cruise some bimbos afterwards, whaddaya say?
1000 L203258 Actually, uh, people seem to think the scar "humanizes" you.
2000 L201064 You picked a helluva day to leave your beeper off!
3000 L433608 I'm just saying she wasn't scared.
4000 L585407 Here we are. You'll be all right here, Miss.
5000 L68123 Oh, God, Basquiat, you scared the shit out of me. How the fuck could you do that to yourself?
6000 L127556 Of course I'm scared.
7000 L533009 To the Brick Kremlin!
8000 L608107 Tomorrow...
9000 L190083 Know what I always thought I wanted outta life, Erin?  A Jaguar.


In [16]:
sentence = "I like making bots."
picked = nns.nearest(sentence_mean(nlp, sentence), 5)[0]
response_line_id = responses[picked]

print("Your line:\n\t", sentence)
print("Most similar turn:\n\t", movie_lines[picked])
print("Response to most similar turn:\n\t", movie_lines[response_line_id])

Your line:
	 I like making bots.
Most similar turn:
	 I like it.
Response to most similar turn:
	 Up here in the hills, a man gets a hankering to smell powder.


In [17]:
!pip install https://github.com/aparrish/semanticsimilaritychatbot/archive/master.zip

     \ 102kB 1.2MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-ykpmxtrk/wheels/f7/af/8e/8a8fbef31bfbfc3b935425efa03db03825795d85f4e23f8255
Successfully built semanticsimilaritychatbot


In [0]:
from semanticsimilaritychatbot import SemanticSimilarityChatbot
chatbot = SemanticSimilarityChatbot(nlp, 300)

In [0]:
sample_n = 10000
for first_id, second_id in random.sample(list(responses.items()), sample_n):
    chatbot.add_pair(movie_lines[first_id], movie_lines[second_id])
chatbot.build()

In [20]:
print(chatbot.response_for("Hello computer!"))

I heard you were looking for me.


In [21]:
my_turn = "The weather's nice today, don't you think?"
for i in range(5, 51, 5):
    print("picking from", i, "possible responses:")
    print(chatbot.response_for(my_turn, i))
    print()

picking from 5 possible responses:
You're one to talk. Still running playback money for the mob.

picking from 10 possible responses:
You, you don't under...

picking from 15 possible responses:
I must have been there fifty times, it's never been locked; they're not suspicious. Everybody who goes there are toffs. They're all into that karma crap: `If I don't harm nobody, nobody harms me' stuff.

picking from 20 possible responses:
Good thing there's not a red line on you, J-man. YEAH!

picking from 25 possible responses:
You, you don't under...

picking from 30 possible responses:
Good deal you were born down here, then.

picking from 35 possible responses:
Fuck me gently with a chainsaw. Do I look like Mother Theresa? If I did, I probably wouldn't mind talking to the Geek Squad.

picking from 40 possible responses:
They're from my old convent.

picking from 45 possible responses:
If it's on the page...

picking from 50 possible responses:
You're one to talk. Still running playback mon

In [0]:
chatbot.save("movielines-10k-sample")

In [0]:
chatbot = SemanticSimilarityChatbot.load("movielines-10k-sample", nlp)

In [24]:
print(chatbot.response_for("I'm going to go get some coffee."))

What are you tired from?


In [0]:
from google.colab import files
files.download('movielines-10k-sample.annoy')
files.download('movielines-10k-sample-data.pkl')
files.download('movielines-10k-sample-chatbot.pkl')

In [0]:
chatbot_html = """
<style type="text/css">#log p { margin: 5px; font-family: sans-serif; }</style>
<div id="log"
     style="box-sizing: border-box;
            width: 600px;
            height: 32em;
            border: 1px grey solid;
            padding: 2px;
            overflow: scroll;">
</div>
<input type="text" id="typehere" placeholder="type here!"
       style="box-sizing: border-box;
              width: 600px;
              margin-top: 5px;">
<script>
function paraWithText(t) {
    let tn = document.createTextNode(t);
    let ptag = document.createElement('p');
    ptag.appendChild(tn);
    return ptag;
}
document.querySelector('#typehere').onchange = async function() {
    let inputField = document.querySelector('#typehere');
    let val = inputField.value;
    inputField.value = "";
    let resp = await getResp(val);
    let objDiv = document.getElementById("log");
    objDiv.appendChild(paraWithText('😀: ' + val));
    objDiv.appendChild(paraWithText('🤖: ' + resp));
    objDiv.scrollTop = objDiv.scrollHeight;
};
async function colabGetResp(val) {
    let resp = await google.colab.kernel.invokeFunction(
        'notebook.get_response', [val], {});
    return resp.data['application/json']['result'];
}
async function webGetResp(val) {
    let resp = await fetch("/response.json?sentence=" + 
        encodeURIComponent(val));
    let data = await resp.json();
    return data['result'];
}
</script>
"""

In [29]:
import IPython
from google.colab import output

display(IPython.display.HTML(chatbot_html + \
                             "<script>let getResp = colabGetResp;</script>"))

def get_response(val):
    resp = chatbot.response_for(val)
    return IPython.display.JSON({'result': resp})

output.register_callback('notebook.get_response', get_response)